<a name="top"></a>
# optunaのTutorial :
- [TutorialのURL](https://optuna.readthedocs.io/en/stable/tutorial/index.html)
- [GitHub](https://github.com/pfnet/optuna)
- [First Optimization](#first_optimization)
    - [二次関数の最小化](#quadratic)
- [Advanced Configurations](#advanced_configurations)
    - [scikit-learnの実装例 (Irisデータ)](#sklearn_iris)
    - [xgboostの実装例 (cancerデータ)](#xgboost_cancer)
- [Pruning Unpromising Trials](#pruning)
    - [scikit-learnの実装例](#sklearn_pruning)
    - [xgboostの実装例](#xgboost_pruning)

<a name="first_optimization"></a>
## First Optimization
- [公式サイト](https://optuna.readthedocs.io/en/stable/tutorial/first.html)

In [1]:
import optuna

<a name="quadratic"></a>
### 二次関数の最小値を探索する最適化問題
- `objective`で定義される二次関数の最小値を探索する
- 最適値は`x=2`

#### 最小化したい目的関数の定義
- 最適化の過程で`suggest_uniform`が呼ばれる
- trial.suggest_uniform は`('name', range_min, range_max)`で指定, 'name'はxである必要はない
- `return`で目的関数を返す

In [2]:
def objective(trial):
    """ 目的関数 """
    x = trial.suggest_uniform('x', -10, 10)
    return (x - 2) ** 2

#### 最適化
- `study`オブジェクトを生成して、`optimize`メソッドに目的関数`objective`を渡す

In [3]:
study = optuna.create_study()
study.optimize(objective, n_trials=100)

[I 2018-12-05 03:57:10,830] Finished a trial resulted in value: 8.090362528567963. Current best value is 8.090362528567963 with parameters: {'x': -0.8443562590800688}.
[I 2018-12-05 03:57:10,832] Finished a trial resulted in value: 98.08979376324466. Current best value is 8.090362528567963 with parameters: {'x': -0.8443562590800688}.
[I 2018-12-05 03:57:10,834] Finished a trial resulted in value: 9.389921340182921. Current best value is 8.090362528567963 with parameters: {'x': -0.8443562590800688}.
[I 2018-12-05 03:57:10,836] Finished a trial resulted in value: 96.42136764019976. Current best value is 8.090362528567963 with parameters: {'x': -0.8443562590800688}.
[I 2018-12-05 03:57:10,838] Finished a trial resulted in value: 0.0877042626971673. Current best value is 0.0877042626971673 with parameters: {'x': 2.296149054864552}.
[I 2018-12-05 03:57:10,841] Finished a trial resulted in value: 1.1374335892826002. Current best value is 0.0877042626971673 with parameters: {'x': 2.2961490548

[I 2018-12-05 03:57:11,516] Finished a trial resulted in value: 1.9366679885954416. Current best value is 0.00048148561762163025 with parameters: {'x': 2.0219427805353294}.
[I 2018-12-05 03:57:11,528] Finished a trial resulted in value: 0.5617230938765964. Current best value is 0.00048148561762163025 with parameters: {'x': 2.0219427805353294}.
[I 2018-12-05 03:57:11,539] Finished a trial resulted in value: 3.6121307308512436. Current best value is 0.00048148561762163025 with parameters: {'x': 2.0219427805353294}.
[I 2018-12-05 03:57:11,550] Finished a trial resulted in value: 1.9973549649470117. Current best value is 0.00048148561762163025 with parameters: {'x': 2.0219427805353294}.
[I 2018-12-05 03:57:11,561] Finished a trial resulted in value: 7.51517901598519. Current best value is 0.00048148561762163025 with parameters: {'x': 2.0219427805353294}.


#### 用語について
- Trial: 目的関数について、1回の評価・試行
- Study: 最適化の過程, 探索, trialの集合
- Parameter: 最適化される変数, 上の例では`x`

### study objectのattribute等
- 最適化の結果
- 試行のログ

In [4]:
print('最適化された変数の値')
print(study.best_params)
print('-'*40)
print('目的関数の最適値')
print(study.best_value)
print('-'*40)
print('最適値を達成した試行')
print(study.best_trial)
print('-'*40)
print('全試行回数')
print(len(study.trials))
print('-'*40)
print('全試行')
print(study.trials)

最適化された変数の値
{'x': 2.0219427805353294}
----------------------------------------
目的関数の最適値
0.00048148561762163025
----------------------------------------
最適値を達成した試行
FrozenTrial(trial_id=42, state=<TrialState.COMPLETE: 1>, value=0.00048148561762163025, datetime_start=datetime.datetime(2018, 12, 5, 3, 57, 11, 44024), datetime_complete=datetime.datetime(2018, 12, 5, 3, 57, 11, 47728), params={'x': 2.0219427805353294}, user_attrs={}, system_attrs={}, intermediate_values={}, params_in_internal_repr={'x': 2.0219427805353294})
----------------------------------------
全試行回数
100
----------------------------------------
全試行
[FrozenTrial(trial_id=0, state=<TrialState.COMPLETE: 1>, value=8.090362528567963, datetime_start=datetime.datetime(2018, 12, 5, 3, 57, 10, 829628), datetime_complete=datetime.datetime(2018, 12, 5, 3, 57, 10, 829882), params={'x': -0.8443562590800688}, user_attrs={}, system_attrs={}, intermediate_values={}, params_in_internal_repr={'x': -0.8443562590800688}), FrozenTrial(trial_id

#### 結果を引き継いで再最適化
- `len(study.trials)`の値から継続的に最適化が行われていることが分かる
- `best_params`が改善している

In [5]:
study.optimize(objective, n_trials=100)

[I 2018-12-05 03:57:11,606] Finished a trial resulted in value: 4.712859864216515. Current best value is 0.00048148561762163025 with parameters: {'x': 2.0219427805353294}.
[I 2018-12-05 03:57:11,618] Finished a trial resulted in value: 11.898120623018198. Current best value is 0.00048148561762163025 with parameters: {'x': 2.0219427805353294}.
[I 2018-12-05 03:57:11,629] Finished a trial resulted in value: 4.406783665331058. Current best value is 0.00048148561762163025 with parameters: {'x': 2.0219427805353294}.
[I 2018-12-05 03:57:11,640] Finished a trial resulted in value: 6.146304871405808. Current best value is 0.00048148561762163025 with parameters: {'x': 2.0219427805353294}.
[I 2018-12-05 03:57:11,651] Finished a trial resulted in value: 3.750760713335725. Current best value is 0.00048148561762163025 with parameters: {'x': 2.0219427805353294}.
[I 2018-12-05 03:57:11,663] Finished a trial resulted in value: 11.01071201863862. Current best value is 0.00048148561762163025 with parame

[I 2018-12-05 03:57:12,997] Finished a trial resulted in value: 1.0511557305746542. Current best value is 1.0527750203026138e-08 with parameters: {'x': 1.9998973951745627}.
[I 2018-12-05 03:57:13,015] Finished a trial resulted in value: 3.3324751459987243. Current best value is 1.0527750203026138e-08 with parameters: {'x': 1.9998973951745627}.
[I 2018-12-05 03:57:13,032] Finished a trial resulted in value: 0.33247050413952306. Current best value is 1.0527750203026138e-08 with parameters: {'x': 1.9998973951745627}.
[I 2018-12-05 03:57:13,050] Finished a trial resulted in value: 0.7595183302564733. Current best value is 1.0527750203026138e-08 with parameters: {'x': 1.9998973951745627}.
[I 2018-12-05 03:57:13,067] Finished a trial resulted in value: 0.676509948157123. Current best value is 1.0527750203026138e-08 with parameters: {'x': 1.9998973951745627}.
[I 2018-12-05 03:57:13,085] Finished a trial resulted in value: 4.73865128211823. Current best value is 1.0527750203026138e-08 with par

In [6]:
print('全試行回数')
print(len(study.trials))
print('-'*40)
print('最適化された変数の値')
print(study.best_params)

全試行回数
200
----------------------------------------
最適化された変数の値
{'x': 1.9998973951745627}


#### 試行回数ではなくtimeoutの時間を指定して実行

In [7]:
study.optimize(objective, timeout=2.0)

[I 2018-12-05 03:57:13,127] Finished a trial resulted in value: 0.00024876013444282525. Current best value is 1.0527750203026138e-08 with parameters: {'x': 1.9998973951745627}.
[I 2018-12-05 03:57:13,146] Finished a trial resulted in value: 0.1699228993850685. Current best value is 1.0527750203026138e-08 with parameters: {'x': 1.9998973951745627}.
[I 2018-12-05 03:57:13,165] Finished a trial resulted in value: 0.30561231754959073. Current best value is 1.0527750203026138e-08 with parameters: {'x': 1.9998973951745627}.
[I 2018-12-05 03:57:13,182] Finished a trial resulted in value: 1.0528011499782632. Current best value is 1.0527750203026138e-08 with parameters: {'x': 1.9998973951745627}.
[I 2018-12-05 03:57:13,200] Finished a trial resulted in value: 0.9233263298154628. Current best value is 1.0527750203026138e-08 with parameters: {'x': 1.9998973951745627}.
[I 2018-12-05 03:57:13,220] Finished a trial resulted in value: 3.0484800501318667. Current best value is 1.0527750203026138e-08 w

In [8]:
print('全試行回数')
print(len(study.trials))
print('-'*40)
print('最適化された変数の値')
print(study.best_params)

全試行回数
294
----------------------------------------
最適化された変数の値
{'x': 1.9998973951745627}


<a name="advanced_configurations"></a>
## Advanced Configurations
- [公式サイト](https://optuna.readthedocs.io/en/stable/tutorial/configurations.html)

### 探索するパラメータ空間の定義方法
- 提示される値は`Sampler`によってサンプリングされる
    - RandomSampler
    - TPESampler

In [9]:
def objective(trial):
    # カテゴリ変数を提示する, (name, choices) choicesにリスト[]で指定
    optimizer = trial.suggest_categorical('optimizer', ['MomentumSGD', 'Adam'])

    # 整数値変数[low, high]から提示, (name, low, high)
    num_layers = trial.suggest_int('num_layers', 1, 3)

    # 連続値変数[low, high)から提示, (name, low, high)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 1.0)

    # 連続値変数, 対数範囲[low, high)から提示 (name, low, high)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)

    # 離散値[low, high, step]から提示, (name, low, high, q)
    drop_path_rate = trial.suggest_discrete_uniform('drop_path_rate', 0.0, 1.0, 0.1)


### 変数の値によって条件分岐して探索することも可能

In [10]:
def objective(trial):
    classifier_name = trial.suggest_categorical('classifier', ['SVC', 'RandomForest'])
    
    # アルゴリズムごとに探索空間を定義
    if classifier_name == 'SVC':
        svc_c = trial.suggest_loguniform('svc_c', 1e-10, 1e10)
        classifier_obj = sklearn.svm.SVC(C=svc_c)
    else:
        rf_max_depth = int(trial.suggest_loguniform('rf_max_depth', 2, 32))
        classifier_obj = sklearn.ensemble.RandomForestClassifier(max_depth=rf_max_depth)


### ループ処理を入れることも可能

In [11]:
def create_model(trial):
    n_layers = trial.suggest_int('n_layers', 1, 3)

    layers = []
    for i in range(n_layers):
        n_units = int(trial.suggest_loguniform('n_units_l{}'.format(i), 4, 128))
        layers.append(L.Linear(None, n_units))
        layers.append(F.relu)
    layers.append(L.Linear(None, 10))

    return chainer.Sequential(*layers)

### 他の留意事項等
- 変数の数を増やすとおよそ指数的に計算量が増える
- `optimize`に渡す変数の中で計算時間にかかわるものとして、`n_trials`と`timeout`がある
- `n_trials`と`timeout`が指定されないと、`Ctrl+C`等で中断されるまで実行される

<a name="sklearn_iris"></a>
### scikit-learnの実装例 (Iris)
- [コードのソース(github)](https://github.com/pfnet/optuna/blob/master/examples/sklearn_simple.py)

In [12]:
import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection
import sklearn.svm


def objective(trial):
    # Iris データの読み込み
    iris = sklearn.datasets.load_iris()
    x, y = iris.data, iris.target

    # 分類器の名前をカテゴリ変数として探索空間を定義
    classifier_name = trial.suggest_categorical('classifier', ['SVC', 'RandomForest'])
    
    # 分類器ごとに変数の探索空間を定義
    if classifier_name == 'SVC':
        svc_c = trial.suggest_loguniform('svc_c', 1e-10, 1e10)
        classifier_obj = sklearn.svm.SVC(C=svc_c)
    else:
        rf_max_depth = int(trial.suggest_loguniform('rf_max_depth', 2, 32))
        classifier_obj = sklearn.ensemble.RandomForestClassifier(max_depth=rf_max_depth)

    # 最適化する目的関数の定義
    score = sklearn.model_selection.cross_val_score(classifier_obj, x, y, n_jobs=-1)
    accuracy = score.mean()
    return 1.0 - accuracy

In [13]:
study = optuna.create_study()
study.optimize(objective, n_trials=100)
print(study.best_trial)

/Users/ikokpro/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[I 2018-12-05 03:57:17,920] Finished a trial resulted in value: 0.04003267973856206. Current best value is 0.04003267973856206 with parameters: {'classifier': 'RandomForest', 'rf_max_depth': 12.989749976030364}.
/Users/ikokpro/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[I 2018-12-05 03:57:19,124] Finished a trial resulted in value: 0.06576797385620914. Current best value is 0.04003267973856206 with parameters: {'classifier': 'RandomFore

[I 2018-12-05 03:57:21,621] Finished a trial resulted in value: 0.060049019607843146. Current best value is 0.0326797385620915 with parameters: {'classifier': 'SVC', 'svc_c': 184.589902309806}.
/Users/ikokpro/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[I 2018-12-05 03:57:21,640] Finished a trial resulted in value: 0.046160130718954306. Current best value is 0.0326797385620915 with parameters: {'classifier': 'SVC', 'svc_c': 184.589902309806}.
/Users/ikokpro/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWar

[I 2018-12-05 03:57:22,181] Finished a trial resulted in value: 0.026552287581699363. Current best value is 0.013480392156862808 with parameters: {'classifier': 'SVC', 'svc_c': 1.9983676575923972}.
/Users/ikokpro/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[I 2018-12-05 03:57:22,197] Finished a trial resulted in value: 0.060049019607843146. Current best value is 0.013480392156862808 with parameters: {'classifier': 'SVC', 'svc_c': 1.9983676575923972}.
/Users/ikokpro/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, F

[I 2018-12-05 03:57:22,789] Finished a trial resulted in value: 0.03962418300653603. Current best value is 0.013480392156862808 with parameters: {'classifier': 'SVC', 'svc_c': 1.9983676575923972}.
/Users/ikokpro/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[I 2018-12-05 03:57:22,810] Finished a trial resulted in value: 0.03308823529411764. Current best value is 0.013480392156862808 with parameters: {'classifier': 'SVC', 'svc_c': 1.9983676575923972}.
/Users/ikokpro/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, Fut

FrozenTrial(trial_id=36, state=<TrialState.COMPLETE: 1>, value=0.013480392156862808, datetime_start=datetime.datetime(2018, 12, 5, 3, 57, 21, 976644), datetime_complete=datetime.datetime(2018, 12, 5, 3, 57, 21, 989797), params={'classifier': 'SVC', 'svc_c': 1.9983676575923972}, user_attrs={}, system_attrs={}, intermediate_values={}, params_in_internal_repr={'classifier': 0, 'svc_c': 1.9983676575923972})


<a name="xgboost_cancer"></a>
### xgboostの実装例 (cancer)
- [コードのソース(github)](https://github.com/pfnet/optuna/blob/master/examples/xgboost_simple.py)

In [14]:
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb


def objective(trial):
    # cancer データの読み込み
    (data, target) = sklearn.datasets.load_breast_cancer(return_X_y=True)
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.25)
    
    # xgb.DMatrixに変換
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dtest = xgb.DMatrix(test_x, label=test_y)

    # 全体で共通の探索領域の定義
    n_round = trial.suggest_int('n_round', 1, 9)
    param = {'silent': 1, 'objective': 'binary:logistic',
             'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
             'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
             'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0)
             }
    
    # アルゴリズムごとの探索領域の定義
    if param['booster'] == 'gbtree' or param['booster'] == 'dart':
        param['max_depth'] = trial.suggest_int('max_depth', 1, 9)
        param['ets'] = trial.suggest_loguniform('eta', 1e-8, 1.0)
        param['gamma'] = trial.suggest_loguniform('gamma', 1e-8, 1.0)
        param['grow_policy'] = trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide'])
    if param['booster'] == 'dart':
        param['sample_type'] = trial.suggest_categorical('sample_type', ['uniform', 'weighted'])
        param['normalize_type'] = trial.suggest_categorical('normalize_type', ['tree', 'forest'])
        param['rate_drop'] = trial.suggest_loguniform('rate_drop', 1e-8, 1.0)
        param['skip_drop'] = trial.suggest_loguniform('skip_drop', 1e-8, 1.0)

    # 学習, 予測, 正答率の計算, 目的関数はaccuracy
    bst = xgb.train(param, dtrain, n_round)
    preds = bst.predict(dtest)
    pred_labels = np.rint(preds) # 四捨五入に近い関数
    accuracy = sklearn.metrics.accuracy_score(test_y, pred_labels)
    return 1.0 - accuracy

In [15]:
study = optuna.create_study()
study.optimize(objective, n_trials=100)
print(study.best_trial)

[I 2018-12-05 03:57:23,231] Finished a trial resulted in value: 0.05594405594405594. Current best value is 0.05594405594405594 with parameters: {'n_round': 3, 'booster': 'dart', 'lambda': 1.6294842684445261e-07, 'alpha': 0.003881620167277564, 'max_depth': 2, 'eta': 0.019729719036663394, 'gamma': 0.0004913017308786807, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 3.9517298923244786e-07, 'skip_drop': 0.005370083273604612}.
[I 2018-12-05 03:57:23,241] Finished a trial resulted in value: 0.14685314685314688. Current best value is 0.05594405594405594 with parameters: {'n_round': 3, 'booster': 'dart', 'lambda': 1.6294842684445261e-07, 'alpha': 0.003881620167277564, 'max_depth': 2, 'eta': 0.019729719036663394, 'gamma': 0.0004913017308786807, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 3.9517298923244786e-07, 'skip_drop': 0.005370083273604612}.
[I 2018-12-05 03:57:23,261] Finished a trial res

[I 2018-12-05 03:57:24,584] Finished a trial resulted in value: 0.027972027972028024. Current best value is 0.020979020979020935 with parameters: {'n_round': 7, 'booster': 'gbtree', 'lambda': 4.525482517272851e-05, 'alpha': 0.0002582716925167782, 'max_depth': 8, 'eta': 0.5380849881804378, 'gamma': 0.0030370634700011136, 'grow_policy': 'lossguide'}.
[I 2018-12-05 03:57:24,609] Finished a trial resulted in value: 0.0699300699300699. Current best value is 0.020979020979020935 with parameters: {'n_round': 7, 'booster': 'gbtree', 'lambda': 4.525482517272851e-05, 'alpha': 0.0002582716925167782, 'max_depth': 8, 'eta': 0.5380849881804378, 'gamma': 0.0030370634700011136, 'grow_policy': 'lossguide'}.
[I 2018-12-05 03:57:24,656] Finished a trial resulted in value: 0.034965034965035. Current best value is 0.020979020979020935 with parameters: {'n_round': 7, 'booster': 'gbtree', 'lambda': 4.525482517272851e-05, 'alpha': 0.0002582716925167782, 'max_depth': 8, 'eta': 0.5380849881804378, 'gamma': 0.00

[I 2018-12-05 03:57:27,030] Finished a trial resulted in value: 0.027972027972028024. Current best value is 0.006993006993006978 with parameters: {'n_round': 4, 'booster': 'gbtree', 'lambda': 0.000282767605649504, 'alpha': 0.000151049118132311, 'max_depth': 5, 'eta': 0.008258546943256029, 'gamma': 0.001268546012115243, 'grow_policy': 'lossguide'}.
[I 2018-12-05 03:57:27,110] Finished a trial resulted in value: 0.06293706293706292. Current best value is 0.006993006993006978 with parameters: {'n_round': 4, 'booster': 'gbtree', 'lambda': 0.000282767605649504, 'alpha': 0.000151049118132311, 'max_depth': 5, 'eta': 0.008258546943256029, 'gamma': 0.001268546012115243, 'grow_policy': 'lossguide'}.
[I 2018-12-05 03:57:27,150] Finished a trial resulted in value: 0.0699300699300699. Current best value is 0.006993006993006978 with parameters: {'n_round': 4, 'booster': 'gbtree', 'lambda': 0.000282767605649504, 'alpha': 0.000151049118132311, 'max_depth': 5, 'eta': 0.008258546943256029, 'gamma': 0.00

FrozenTrial(trial_id=53, state=<TrialState.COMPLETE: 1>, value=0.006993006993006978, datetime_start=datetime.datetime(2018, 12, 5, 3, 57, 24, 871168), datetime_complete=datetime.datetime(2018, 12, 5, 3, 57, 24, 907850), params={'n_round': 4, 'booster': 'gbtree', 'lambda': 0.000282767605649504, 'alpha': 0.000151049118132311, 'max_depth': 5, 'eta': 0.008258546943256029, 'gamma': 0.001268546012115243, 'grow_policy': 'lossguide'}, user_attrs={}, system_attrs={}, intermediate_values={}, params_in_internal_repr={'n_round': 4, 'booster': 0, 'lambda': 0.000282767605649504, 'alpha': 0.000151049118132311, 'max_depth': 5, 'eta': 0.008258546943256029, 'gamma': 0.001268546012115243, 'grow_policy': 1})


<a name="pruning"></a>
## Pruning Unpromising Trials
- [公式サイト](https://optuna.readthedocs.io/en/stable/tutorial/pruning.html)

### 有望ではない学習を途中で打ち切る
- 繰り返し最適化(学習)のloop内に`report()`と`should_prune()`を入れる
- ログを見ると、`TrialState.PRUNED`と書かれたステップがあり、そこでは指定回数(今回は100回)の学習を終える前に学習が打ち切られている

<a name="sklearn_pruning"></a>
### scikit-learnの実装例 (Iris)

In [16]:
import sklearn.datasets
import sklearn.linear_model
import sklearn.model_selection

import optuna

def objective(trial):
    
    # Irisデータの読み込み
    iris = sklearn.datasets.load_iris()
    classes = list(set(iris.target))
    train_x, test_x, train_y, test_y = \
        sklearn.model_selection.train_test_split(iris.data, iris.target, test_size=0.25, random_state=0)

    # 学習率のサンプリング範囲を定義
    alpha = trial.suggest_loguniform('alpha', 1e-5, 1e-1)
    
    # 学習モデルを作成
    clf = sklearn.linear_model.SGDClassifier(alpha=alpha)

    # 繰り返し学習
    for step in range(100):
        clf.partial_fit(train_x, train_y, classes=classes)

        # 学習途中の目的関数の値を report する
        intermediate_value = 1.0 - clf.score(test_x, test_y)
        trial.report(intermediate_value, step)

        # 学習途中の目的関数値にもとづいて打ち切りを判断する
        if trial.should_prune(step):
            raise optuna.structs.TrialPruned()

    return 1.0 - clf.score(test_x, test_y)

# 打ち切りの基準として median stopping を指定
study = optuna.create_study(pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=20)

[I 2018-12-05 03:57:27,794] Finished a trial resulted in value: 0.02631578947368418. Current best value is 0.02631578947368418 with parameters: {'alpha': 0.0013725045527894573}.
[I 2018-12-05 03:57:27,909] Finished a trial resulted in value: 0.10526315789473684. Current best value is 0.02631578947368418 with parameters: {'alpha': 0.0013725045527894573}.
[I 2018-12-05 03:57:28,037] Finished a trial resulted in value: 0.23684210526315785. Current best value is 0.02631578947368418 with parameters: {'alpha': 0.0013725045527894573}.
[I 2018-12-05 03:57:28,184] Finished a trial resulted in value: 0.052631578947368474. Current best value is 0.02631578947368418 with parameters: {'alpha': 0.0013725045527894573}.
[I 2018-12-05 03:57:28,352] Finished a trial resulted in value: 0.21052631578947367. Current best value is 0.02631578947368418 with parameters: {'alpha': 0.0013725045527894573}.
[I 2018-12-05 03:57:28,689] Finished a trial resulted in value: 0.02631578947368418. Current best value is 0.

<a name="xgboost_pruning"></a>
### xgboost等にも用いることが出来る
- 例は上で用いたcancerのデータ
- [コードのソース(github)](https://github.com/pfnet/optuna/blob/master/examples/pruning/xgboost_integration.py)

In [17]:
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb


def objective(trial):
    data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.25)
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dtest = xgb.DMatrix(test_x, label=test_y)

    n_round = trial.suggest_int('n_round', 1, 9)
    param = {'silent': 1, 'objective': 'binary:logistic',
             'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
             'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
             'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0)
             }

    if param['booster'] == 'gbtree' or param['booster'] == 'dart':
        param['max_depth'] = trial.suggest_int('max_depth', 1, 9)
        param['ets'] = trial.suggest_loguniform('eta', 1e-8, 1.0)
        param['gamma'] = trial.suggest_loguniform('gamma', 1e-8, 1.0)
        param['grow_policy'] = trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide'])
    if param['booster'] == 'dart':
        param['sample_type'] = trial.suggest_categorical('sample_type', ['uniform', 'weighted'])
        param['normalize_type'] = trial.suggest_categorical('normalize_type', ['tree', 'forest'])
        param['rate_drop'] = trial.suggest_loguniform('rate_drop', 1e-8, 1.0)
        param['skip_drop'] = trial.suggest_loguniform('skip_drop', 1e-8, 1.0)

    # 打ち切りを判断するためにcallbackを加える
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation-error')
    bst = xgb.train(param, dtrain, n_round, evals=[(dtest, 'validation')],
                    callbacks=[pruning_callback])
    preds = bst.predict(dtest)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(test_y, pred_labels)
    return 1.0 - accuracy

In [18]:
study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
study.optimize(objective, n_trials=100)
print(study.best_trial)

[0]	validation-error:0.146853
[1]	validation-error:0.153846
[2]	validation-error:0.146853
[3]	validation-error:0.153846
[4]	validation-error:0.153846


[I 2018-12-05 03:57:30,400] Finished a trial resulted in value: 0.15384615384615385. Current best value is 0.15384615384615385 with parameters: {'n_round': 5, 'booster': 'gblinear', 'lambda': 0.955248094300941, 'alpha': 3.142815417273797e-05}.


[0]	validation-error:0.062937
[1]	validation-error:0.062937
[2]	validation-error:0.048951
[3]	validation-error:0.090909
[4]	validation-error:0.048951
[5]	validation-error:0.055944
[6]	validation-error:0.055944
[7]	validation-error:0.06993


[I 2018-12-05 03:57:30,435] Finished a trial resulted in value: 0.05594405594405594. Current best value is 0.05594405594405594 with parameters: {'n_round': 8, 'booster': 'dart', 'lambda': 0.00031951729848139814, 'alpha': 6.517255006264683e-05, 'max_depth': 9, 'eta': 0.7918839188228545, 'gamma': 1.1579451296710638e-06, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 0.4670139563805792, 'skip_drop': 0.11681444363432103}.


[0]	validation-error:0.041958
[1]	validation-error:0.027972
[2]	validation-error:0.034965
[3]	validation-error:0.020979
[4]	validation-error:0.020979
[5]	validation-error:0.013986
[6]	validation-error:0.013986
[7]	validation-error:0.013986
[8]	validation-error:0.013986


...

[0]	validation-error:0.132867
[1]	validation-error:0.104895
[2]	validation-error:0.111888
[3]	validation-error:0.097902
[4]	validation-error:0.097902
[5]	validation-error:0.097902


[I 2018-12-05 03:57:35,414] Setting trial status as TrialState.PRUNED. Trial was pruned at iteration 6.


[0]	validation-error:0.132867
[1]	validation-error:0.111888
[2]	validation-error:0.104895
[3]	validation-error:0.097902
[4]	validation-error:0.090909
[5]	validation-error:0.090909


[I 2018-12-05 03:57:35,485] Setting trial status as TrialState.PRUNED. Trial was pruned at iteration 6.


FrozenTrial(trial_id=2, state=<TrialState.COMPLETE: 1>, value=0.013986013986013957, datetime_start=datetime.datetime(2018, 12, 5, 3, 57, 30, 437089), datetime_complete=datetime.datetime(2018, 12, 5, 3, 57, 30, 472497), params={'n_round': 9, 'booster': 'dart', 'lambda': 0.026047313347139816, 'alpha': 7.88394249425758e-07, 'max_depth': 9, 'eta': 0.00013609586584108728, 'gamma': 0.013765350901623459, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 8.326865238773264e-08, 'skip_drop': 0.46866578470656534}, user_attrs={}, system_attrs={}, intermediate_values={0: 0.041958, 1: 0.027972, 2: 0.034965, 3: 0.020979, 4: 0.020979, 5: 0.013986, 6: 0.013986, 7: 0.013986, 8: 0.013986}, params_in_internal_repr={'n_round': 9, 'booster': 2, 'lambda': 0.026047313347139816, 'alpha': 7.88394249425758e-07, 'max_depth': 9, 'eta': 0.00013609586584108728, 'gamma': 0.013765350901623459, 'grow_policy': 0, 'sample_type': 0, 'normalize_type': 0, 'rate_drop': 8.32686523877

#### [トップへ](#top)